In [ ]:
import numpy as np
import pandas as pd
# Load datasets (use your own paths)
mh_precip = pd.read_csv("../data/raw/MH_precipitation.csv")
mh_temp = pd.read_csv("../data/raw/MH_temperature.csv")
mp_precip = pd.read_csv("../data/raw/MP_precipitation.csv")
mp_temp = pd.read_csv("../data/raw/MP_temperature.csv")

### Rainfall–Yield Proxy

- **Significant rainfall anomalies (drought or excessive rain) directly affect crop yields.**
- We can detect years of **deficient rainfall** (e.g., rainfall < 80% of mean seasonal rainfall) and **excess rainfall** (e.g., >120% of mean).
- Using **literature-based yield reduction factors**, we can estimate potential economic losses.

In [ ]:

# --- Seasonal rainfall for MH (Kharif and Rabi) ---
def seasonal_rainfall(df):
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    kharif = df[df['month'].isin([6,7,8,9,10])].groupby('year')['rainfall_mm'].mean()
    rabi = df[df['month'].isin([11,12,1,2,3])].groupby('year')['rainfall_mm'].mean()
    return kharif, rabi

mh_k_rain, mh_r_rain = seasonal_rainfall(mh_precip)
mp_k_rain, mp_r_rain = seasonal_rainfall(mp_precip)

# --- Identify drought/excess years ---
def classify_rainfall_anomaly(seasonal_rain):
    mean_rain = seasonal_rain.mean()
    return seasonal_rain.apply(lambda x: 
        "Drought" if x < 0.8 * mean_rain else ("Excess Rain" if x > 1.2 * mean_rain else "Normal"))

mh_kharif_status = classify_rainfall_anomaly(mh_k_rain)
mp_kharif_status = classify_rainfall_anomaly(mp_k_rain)

# --- Assign approximate yield impact (%) ---
impact_map = {"Drought": -20, "Excess Rain": -10, "Normal": 0}
mh_kharif_impact = mh_kharif_status.map(impact_map)
mp_kharif_impact = mp_kharif_status.map(impact_map)

# --- Economic Proxy Table ---
mh_economic_proxy = pd.DataFrame({
    "Kharif_Rainfall_mm": mh_k_rain,
    "Kharif_Status": mh_kharif_status,
    "Yield_Impact_%": mh_kharif_impact
})
print("MH Economic Proxy:\n", mh_economic_proxy.head())

MH Economic Proxy:
       Kharif_Rainfall_mm Kharif_Status  Yield_Impact_%
year                                                  
2010            9.803467        Normal               0
2011            7.395025        Normal               0
2012            9.265423        Normal               0
2013           10.367983   Excess Rain             -10
2014            6.593383        Normal               0


### Economic Impact Estimation Workflow

1. **Classifies years into “Drought,” “Excess Rain,” or “Normal”**  
   - Based on seasonal rainfall data and thresholds (e.g., rainfall < 80% of mean = Drought).

2. **Assumes crop area, price, and yield**  
   - These parameters (area in hectares, price in INR/quintal, yield in quintals/hectare) can be customized for each district or crop.

3. **Calculates baseline revenue**  
   - Baseline Revenue = **Yield × Price × Area**.

4. **Applies yield loss percentage**  
   - -20% reduction for **Drought** years.  
   - -10% reduction for **Excess Rain** years.  
   - **0% change** for Normal years.

5. **Outputs a table with estimated economic loss per crop per year**  
   - Provides columns for Year, Crop, Rainfall Status, Base Yield (qtl), Base Revenue (INR), and Estimated Loss (INR).

In [6]:
import pandas as pd
import numpy as np

# --- Step 1: Ensure datetime format ---
for df in [mh_precip, mp_precip, mh_temp, mp_temp]:
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

# --- Step 2: Seasonal Rainfall Calculation ---
def seasonal_rainfall(df):
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    kharif = df[df['month'].isin([6, 7, 8, 9, 10])].groupby('year')['rainfall_mm'].mean()
    rabi = df[df['month'].isin([11, 12, 1, 2, 3])].groupby('year')['rainfall_mm'].mean()
    return kharif, rabi

mh_k_rain, mh_r_rain = seasonal_rainfall(mh_precip)
mp_k_rain, mp_r_rain = seasonal_rainfall(mp_precip)

# --- Step 3: Classify Rainfall Anomalies ---
def classify_rainfall_anomaly(seasonal_rain):
    mean_rain = seasonal_rain.mean()
    return seasonal_rain.apply(lambda x: 
        "Drought" if x < 0.8 * mean_rain else ("Excess Rain" if x > 1.2 * mean_rain else "Normal"))

mh_k_status = classify_rainfall_anomaly(mh_k_rain)
mp_k_status = classify_rainfall_anomaly(mp_k_rain)

# --- Step 4: Define Crop Area and Prices (Assumptions) ---
# Units: area (hectares), price (INR per quintal), yield (quintal per hectare)
crop_info = {
    "Soybean": {"area": 3000000, "price": 4000, "yield": 10},
    "Cotton":  {"area": 2500000, "price": 6000, "yield": 8},
    "Wheat":   {"area": 2000000, "price": 2500, "yield": 25},
    "Gram":    {"area": 1500000, "price": 5000, "yield": 8},
    "Paddy":   {"area": 1800000, "price": 2200, "yield": 20},
}

# --- Step 5: Estimate Yield Impact and Loss ---
impact_map = {"Drought": -20, "Excess Rain": -10, "Normal": 0}  # % yield reduction

def estimate_losses(seasonal_status, crops):
    loss_data = []
    for year, status in seasonal_status.items():
        for crop, vals in crops.items():
            yield_loss_pct = impact_map[status]
            base_yield = vals['area'] * vals['yield']
            base_revenue = base_yield * vals['price']
            estimated_loss = (yield_loss_pct / 100) * base_revenue
            loss_data.append([year, crop, status, base_yield, base_revenue, estimated_loss])
    return pd.DataFrame(loss_data, columns=['Year', 'Crop', 'Season_Status', 'Base_Yield(qtl)', 'Base_Revenue(INR)', 'Estimated_Loss(INR)'])

# Calculate losses for MH (Kharif crops: Soybean, Cotton)
mh_losses = estimate_losses(mh_k_status, {"Soybean": crop_info["Soybean"], "Cotton": crop_info["Cotton"]})

# Calculate losses for MP (Kharif crops: Soybean, Paddy)
mp_losses = estimate_losses(mp_k_status, {"Soybean": crop_info["Soybean"], "Paddy": crop_info["Paddy"]})

# --- Step 6: Combine Results ---
economic_impact = pd.concat([mh_losses, mp_losses], keys=['Maharashtra', 'Madhya Pradesh'])

# Display first few rows
economic_impact.head(10)

Year     Crop Season_Status  Base_Yield(qtl)  \
Maharashtra 0  2010  Soybean        Normal         30000000   
            1  2010   Cotton        Normal         20000000   
            2  2011  Soybean        Normal         30000000   
            3  2011   Cotton        Normal         20000000   
            4  2012  Soybean        Normal         30000000   
            5  2012   Cotton        Normal         20000000   
            6  2013  Soybean   Excess Rain         30000000   
            7  2013   Cotton   Excess Rain         20000000   
            8  2014  Soybean        Normal         30000000   
            9  2014   Cotton        Normal         20000000   

               Base_Revenue(INR)  Estimated_Loss(INR)  
Maharashtra 0       120000000000         0.000000e+00  
            1       120000000000         0.000000e+00  
            2       120000000000         0.000000e+00  
            3       120000000000         0.000000e+00  
            4       120000000000         0.000000e+00  
            5       120000000000         0.000000e+00  
            6       120000000000        -1.200000e+10  
            7       120000000000        -1.200000e+10  
            8       120000000000         0.000000e+00  
            9       120000000000         0.000000e+00